# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.2.0
['.config', 'weights', 'model_name', 'tmp', 'models', 'sample_data']


## Task 2: Create Model

In [2]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.reshape(x_train, (x_train.shape[0], 784))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255.

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

4423680/4422102 [==============================] - 0s 0us/step


## Task 4: Model Checkpoint During Training

In [4]:
checkpoint_dir = 'weights/'

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}'),
            monitor='val_acc', save_weights_only=True, save_best_only=True
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 11ms/step - loss: 0.7030 - acc: 0.7624 - val_loss: 0.4980 - val_acc: 0.8212
Epoch 2/2
118/118 [==============================] - 1s 8ms/step - loss: 0.4327 - acc: 0.8471 - val_loss: 0.4334 - val_acc: 0.8456


In [5]:
os.listdir(checkpoint_dir)

['epoch_01_acc_0.8212.data-00000-of-00001',
 'checkpoint',
 'epoch_02_acc_0.8456.index',
 'epoch_01_acc_0.8212.index',
 'epoch_02_acc_0.8456.data-00000-of-00001']

## Task 5: Load Weights

In [6]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.353929042816162, 0.08560000360012054]


In [10]:
model.load_weights('weights/epoch_02_acc_0.8456')
print(model.evaluate(x_test, y_test, verbose=False))

[0.43342486023902893, 0.8456000089645386]


## Task 6: Saving Complete Model During Training

In [11]:
models_dir = 'models'

model = create_model()

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(models_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}.h5'),
            monitor='val_acc', save_weights_only=False, save_best_only=False
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 9ms/step - loss: 0.6544 - acc: 0.7811 - val_loss: 0.5206 - val_acc: 0.8139
Epoch 2/2
118/118 [==============================] - 1s 9ms/step - loss: 0.4296 - acc: 0.8490 - val_loss: 0.4328 - val_acc: 0.8452


In [12]:
os.listdir(models_dir)

['epoch_02_acc_0.8452.h5', 'epoch_01_acc_0.8139.h5']

## Task 7: Load Models

In [13]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.3219082355499268, 0.14149999618530273]


In [15]:
model = tf.keras.models.load_model('models/epoch_02_acc_0.8452.h5')
model.summary()
print(model.evaluate(x_test, y_test, verbose=False))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
[0.4328327178955078, 0.8452000021934509]


## Task 8: Manually Saving Weights and Models

In [16]:
model.save_weights('tmp/manually_saved')
print(os.listdir('tmp'))

['checkpoint', 'manually_saved.data-00000-of-00001', 'manually_saved.index']


In [17]:
model.save('tmp/manually_saved_model.h5')
print(os.listdir('tmp'))

['checkpoint', 'manually_saved.data-00000-of-00001', 'manually_saved_model.h5', 'manually_saved.index']


## Task 9: Exporting and Restoring SavedModel Format

In [18]:
model.save('model_name')
print(os.listdir('model_name'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_name/assets
['assets', 'saved_model.pb', 'variables']


In [19]:
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test, y_test, verbose=False))

[0.4328327178955078, 0.8452000021934509]
